# Lab 3: Regularized Linear Regression
Covered Topics:
> Ridge Regression

> SGD for Ridge Regression

> LASSO and Iterative Soft-thresholding

> Elastic Net Regularization

> Iterative Reweighted Least Squares (IRLS)

# Motivation
> Implement the commonly-used regularization methods for learning regression.

> Implement commonly-used learning algorithm, such as soft-thresholding for lasso, and iteratively reweighted least squares (IRLS) for robust linear regression.

> Quantitatively analyze the performance of different methods on model estimation and data prediction.


# Data Simulator and Testing Function (Don't change them)

In [1]:
import numpy as np
# don't add any other packages


# data simulator and testing function (Don't change them)
def linear_data_simulator(n_train: int = 50,
                          n_test: int = 10,
                          dim: int = 10,
                          v_noise: float = 2,
                          prior: str = 'Gauss',
                          noise: str = 'Gauss',
                          r_seed: int = 42) -> dict:
    """
    Simulate the training and testing data generated by a polynomial model
    :param n_train: the number of training data
    :param n_test: the number of testing data
    :param dim: the dimension of feature
    :param v_noise: the hyper-parameter controlling the variance of data noise
    :param prior: the prior distribution of model parameters, Gauss or Laplace
    :param noise: the noise type, Gauss or Laplace
    :param r_seed: the random seed
    :return:
        a dictionary containing training set, testing set, and the ground truth parameters
    """
    x_train = np.random.RandomState(r_seed).rand(n_train, dim)
    x_test = np.random.RandomState(r_seed).rand(n_test, dim)
    if prior == 'Gauss':
        weights = np.random.RandomState(r_seed).randn(dim, 1)
    else:
        weights = np.random.RandomState(r_seed).laplace(
            loc=0, scale=1, size=(dim, 1))
    if noise == 'Gauss':
        y_train = x_train @ weights + v_noise * \
            np.random.RandomState(r_seed).randn(n_train, 1)
        y_test = x_test @ weights + v_noise * \
            np.random.RandomState(r_seed).randn(n_test, 1)
    else:
        y_train = x_train @ weights + \
            np.random.RandomState(r_seed).laplace(
                loc=0, scale=v_noise, size=(n_train, 1))
        y_test = x_test @ weights + \
            np.random.RandomState(r_seed).laplace(
                loc=0, scale=v_noise, size=(n_test, 1))
    data = {'train': [x_train, y_train],
            'test': [x_test, y_test],
            'real': weights}
    return data


def mse(x: np.ndarray, x_est: np.ndarray) -> float:
    return np.sum((x - x_est) ** 2) / x.shape[0]


def testing(x: np.ndarray, y: np.ndarray, weights: np.ndarray) -> float:
    """
    Compute the MSE of regression based on current model
    :param x: testing data with size (N, )
    :param y: testing label with size (N, 1)
    :param weights: model parameter with size (D, 1)
    :return:
        MSE
    """
    y_est = x @ weights
    return mse(y, y_est)

# Task 1
Given the problem
$$
\min_{\boldsymbol{w}} \|\boldsymbol{y} - \boldsymbol{X} \boldsymbol{w}\|_2^2 + \gamma \|\boldsymbol{w}\|_2^2
$$
implement the function “training” to achieve its closed-form solution.

# Solution 1
The closed form solution of **Ridge Regression** is given by
$$
    \dfrac{\partial L}{\partial \boldsymbol{w}}=-2\boldsymbol{X}^\top (\boldsymbol{y}-\boldsymbol{X}\boldsymbol{w})+2\gamma \boldsymbol{w}=0\Rightarrow \boldsymbol{w}=(\boldsymbol{X}^\top \boldsymbol{X}+\gamma \boldsymbol{I})^{-1}\boldsymbol{X}^\top \boldsymbol{y}
$$

In [2]:
def training(x: np.ndarray, y: np.ndarray, gamma: float) -> np.ndarray:
    """
    The training function of ridge regression model

    min_w ||y - Xw||_2^2 + gamma * ||w||_2^2

    :param x: input data with size (N, dim)
    :param y: labels of data with size (N, 1)
    :param gamma: the weight of the ridge regularizer
    :return:
        a weight vector with size (dim, 1)
    """
    inv = np.linalg.inv(x.T @ x + gamma * np.eye(x.shape[1]))
    return inv @ x.T @ y

A little example that examines our correctiveness.

In [3]:
x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([[3], [7], [11]])
gamma = 0.1
# this should be [[1], [1]] because y = x1 + x2

weights = training(x, y, gamma)
print(weights)

[[0.96345515]
 [1.0268801 ]]


# Task 2
Given the problem in (1), implement the function “training_sgd” to learn the model via stochastic gradient descent.

# Solution 2
Similarily, we only need to change the gradient for each iteration in the SGD algorithm.
> Polynomial regression: $\dfrac{\partial L}{\partial \boldsymbol{w}}=2\boldsymbol{X}^\top_B(\boldsymbol{X}_B\boldsymbol{w}_{t-1}-\boldsymbol{y}_B)$.

> Ridge regression: $\dfrac{\partial L}{\partial \boldsymbol{w}}=2\boldsymbol{X}^\top_B(\boldsymbol{X}_B\boldsymbol{w}_{t-1}-\boldsymbol{y}_B)+2\gamma \boldsymbol{w}_{t-1}$.

In [4]:
def training_sgd(x: np.ndarray,
                 y: np.ndarray,
                 gamma: float,
                 epoch: int = 10,
                 batch_size: int = 10,
                 lr: float = 1e-4,
                 r_seed: int = 1) -> np.ndarray:
    """
    The stochastic gradient descent method of ridge regression.
    :param x: input data with size (N, dim)
    :param y: labels of data with size (N, 1)
    :param gamma: the weight of ridge regularizer
    :param epoch: the number of epochs
    :param batch_size: the batch size for sgd
    :param lr: the learning rate
    :param r_seed: random seed
    :return:
        a weight vector with size (order, 1)
    """
    np.random.seed(r_seed)
    # Number of samples
    n = x.shape[0]
    # Dimension of the input
    dim = x.shape[1]
    weights = np.random.randn(dim, 1)
    for e in range(epoch):
        idx = np.random.permutation(n)
        x = x[idx]
        y = y[idx]
        for i in range(0, n, batch_size):
            x_batch = x[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            grad = (-2 * x_batch.T @ (y_batch - x_batch @ weights) + 2 * gamma * weights) / batch_size
            weights -= lr * grad
    return weights


A little example that examines our correctiveness.

In [5]:
weight_sgd = training_sgd(x, y, gamma, epoch=10000)
print("weight: ")
print([str(weight_sgd[0]) for weight_sgd in weight_sgd])
print("estimated y: ")
print([str((x[i] @ weight_sgd)[0]) for i in range(len(x))])
print("real y: ")
print([str((y[i])[0]) for i in range(len(y))])

weight: 
['2.082887016911663', '0.1431523566821604']
estimated y: 
['2.369191730275984', '6.8212704774636315', '11.273349224651277']
real y: 
['3', '7', '11']


# Task 3
Implement the iterative soft-thresholding algorithm to solve the lasso problem
$$
\min_{\boldsymbol{w}} \|\boldsymbol{y} - \boldsymbol{X} \boldsymbol{w}\|_2^2 + \gamma \|\boldsymbol{w}\|_1
$$

# Solution 3
First we investiget the closed-form solution of LASSO Regression under **orthonormal condition**, i.e. $\boldsymbol{X}^\top \boldsymbol{X}=\boldsymbol{I_D}$. This is a special case but serves as a good starting point for understanding the general case.

The closed-form solution of LASSO Regression under **soft-thresholding** can written as
$$
    \begin{align*}
        w_{d}^*=S_{\lambda}(w_{OLS,d})=\text{sign}(w_{OLS,d})\cdot \max\{|w_{OLS,d}|-\lambda,0\},\ d=1,2,\cdots,D
    \end{align*}
$$
with each $w_d^*$ being the $d$-th element of the solution $\boldsymbol{w}^*$. Here, $w_{OLS,d}$ is the $d$-th element of the OLS solution $\boldsymbol{w}_{OLS}=(\boldsymbol{X}^\top \boldsymbol{X})^{-1}\boldsymbol{X}^\top \boldsymbol{y}$.

The $S_{\lambda}(\cdot)$ is the soft-thresholding function with $\lambda$ being the threshold, which we will implement first.

In [17]:
def soft_thresholding(x: np.ndarray, thres: float) -> np.ndarray:
    """
    The soft-thresholding operator
    :param x: input array with arbitrary size
    :param thres: the threshold
    :return:
        sign(x) * max{0, |x|-thres}
    """
    return np.sign(x) * np.maximum(np.abs(x) - thres, 0)

In [16]:
x = np.array([1, 2, -3, 4, 5])
thres = 2
# this should be [0, 0, -1, 2, 3]
soft_thresholding(x, thres)

array([ 0,  0, -1,  2,  3])

More generally, when $\boldsymbol{X}^\top \boldsymbol{X}\ne \boldsymbol{I_D}$, we can construct orthonormal vectors column-wisely and update parameters iteratively. In the $t$-th iteration, it can be proved that the $d$-th parameter can be updated by **soft-thresholding**:
$$
    \begin{align*}
        w_{d}^{(t+1)}=S_{\frac{\lambda}{\|\boldsymbol{x}_d\|_2^2}}\left(\dfrac{\boldsymbol{x}_d^\top (\boldsymbol{y}-\boldsymbol{X}_{-d}\boldsymbol{w}_{-d}^{(t)})}{\|\boldsymbol{x_d}\|_2^2}\right)
    \end{align*}
$$
It's worth noticing that we are actually updating $w_d^{(t+1)}$ with $w_1^{(t+1)},w_2^{(t+1)},\cdots,w_{d-1}^{(t+1)}$ and $w_{d+1}^{(t)},\cdots,w_D^{(t)}$. So the weight vector $\boldsymbol{w}^{(t+1)}$ is updated column-wisely and **non-concurrently**.

In the actual coding implementation, $\boldsymbol{y}-\boldsymbol{X}_{-d}\boldsymbol{w}_{-d}^{(t)}$ is computed by
$$
    \begin{align*}
        \boldsymbol{y}-\boldsymbol{X}_{-d}\boldsymbol{w}_{-d}^{(t)}=\boldsymbol{y}-\boldsymbol{X}\boldsymbol{w}^{(t)}+\boldsymbol{x}_d w_d^{(t)}
    \end{align*}
$$




In [14]:
def training_lasso(x: np.ndarray,
                   y: np.ndarray,
                   gamma: float,
                   iteration: int = 100,
                   r_seed: int = 1) -> np.ndarray:
    """
    The training function of lasso regression model

    min_w ||y - Xw||_2^2 + gamma * ||w||_1

    :param x: input data with size (N, dim)
    :param y: labels of data with size (N, 1)
    :param gamma: the weight of the lasso regularizer
    :param iteration: the number of iterations for soft-thresholding
    :param r_seed: the random seed for initializing model.
    :return:
        a weight vector with size (dim, 1)
    """
    num = x.shape[0]
    dim = x.shape[1]
    np.random.seed(r_seed)
    weights = np.random.randn(dim, 1)
    for t in range(iteration):
        for d in range(dim):
            # x_d is x[:, d] (taking out the d-th column)
            x_d = x[:, d]
            xd2_square = np.linalg.norm(x_d) ** 2
            # threshold is gamma / ||x_d||^2
            threshold = gamma / xd2_square
            # update the d-th weight
            # w_hat = x_d^T(y - x_{-d}w_{-d}) / ||x_d||^2
            #       = x_d^T(y - xw + x_d w_d) / ||x_d||^2
            w_hat = x_d.T @ (y - x @ weights + (x_d * weights[d]).reshape(num,1)) / xd2_square
            # soft-thresholding
            weights[d] = soft_thresholding(w_hat, threshold)
    return weights

In [15]:
x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([[3], [7], [11]])
gamma = 0.1
# this should be [[1], [1]] because y = x1 + x2
weights = training_lasso(x, y, gamma, iteration=10000)
weights

array([[0.95  ],
       [1.0375]])

[Detail]: Don't forget `.reshape(num,1)` in `w_hat = x_d.T @ (y - x @ weights + (x_d * weights[d]).reshape(num,1)) / xd2_square` because the random weight that we generate is (1,D)

# Task 4
Implement an algorithm to solve the linear regression with elastic net regularization
$$
\min_{\boldsymbol{w}} \|\boldsymbol{y} - \boldsymbol{X} \boldsymbol{w}\|_2^2 + \gamma_1 \|\boldsymbol{w}\|_1 + \gamma_2 \|\boldsymbol{w}\|_2^2
$$
(Hint: Reformulate it to Lasso.)

## Solution 4
The basic idea is to integrate the $L_2$ regularization term into MSE $\|\boldsymbol{y-Xw}\|_2^2$ and then apply the iterative soft-thresholding w.r.t. $L_1$ regularization. Note that the term $\lambda_2 \|\boldsymbol{w}\|_2^2$ can be written as 
$$
    \begin{align*}
        \lambda_2 \|\boldsymbol{w}\|_2^2=\|\sqrt{\lambda_2}\boldsymbol{w}\|_2^2=\|\boldsymbol{0}_D-\sqrt{\lambda_2}\boldsymbol{I_D}\boldsymbol{w}\|_2^2
    \end{align*}
$$
where $\boldsymbol{0}_D$ is a $D$-dimensional zero vector and $\boldsymbol{I_D}$ is a $D\times D$ identity matrix. We can therefore integrate the $L_2$ regularization term into the MSE term as
$$ 
    \begin{align*}
        &\|\boldsymbol{y-Xw}\|_2^2+\|\boldsymbol{0}_D-\sqrt{\lambda_2}\boldsymbol{I_D}\boldsymbol{w}\|_2^2+\lambda_1 \|\boldsymbol{w}\|_1\\
        =&\left\|\begin{bmatrix}
            \boldsymbol{y}\\
            \boldsymbol{0}_D
        \end{bmatrix}-\begin{bmatrix}
            \boldsymbol{X}\\
            \sqrt{\lambda_2}\boldsymbol{I_D}
        \end{bmatrix}\boldsymbol{w} \right\|_2^2+\lambda_1\|\boldsymbol{w}\|_1
    \end{align*}
$$
And this goes back to $L_1$ regularization situation and we can put new matrix
$$
    \begin{align*}
        \boldsymbol{X'}=\begin{bmatrix}
            \boldsymbol{X}\\
            \sqrt{\lambda_2}\boldsymbol{I_D}
        \end{bmatrix}, \boldsymbol{y'}=\begin{bmatrix}
            \boldsymbol{y}\\
            \boldsymbol{0}_D
        \end{bmatrix}
    \end{align*}
$$
into the iterative soft-thresholding solver for LASSO.

In [54]:
def training_elastic(x: np.ndarray,
                     y: np.ndarray,
                     gamma1: float,
                     gamma2: float,
                     iteration: int = 100,
                     r_seed: int = 1) -> np.ndarray:
    """
    The training function of lasso regression model

    min_w ||y - Xw||_2^2 + gamma1 * ||w||_1 + gamma2 * ||w||_2^2

    :param x: input data with size (N, dim)
    :param y: labels of data with size (N, 1)
    :param gamma1: the weight of the lasso regularizer
    :param gamma2: the weight of the ridge regularizer
    :param iteration: the number of iterations for soft-thresholding
    :param r_seed: the random seed for initializing model.
    :return:
        a weight vector with size (dim, 1)
    """
    dim = x.shape[1]
    x_concat = np.concatenate((x, np.sqrt(gamma2) * np.eye(dim)), axis=0)
    y_concat = np.concatenate((y, np.zeros(dim).reshape(dim, 1)))
    return training_lasso(x_concat, y_concat, gamma1, iteration, r_seed)

In [55]:

x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([[3], [7], [11]])
gamma1 = 0.1
gamma2 = 0.1
# this should be [[1], [1]] because y = x1 + x2
weights = training_elastic(x, y, gamma1, gamma2, iteration=10000)
weights

array([[0.9269103 ],
       [1.05376019]])

# Task 5
Implement the iteratively reweighted least squares (IRLS) to solve robust linear regression:
$$
\min_{\boldsymbol{w}} \|\boldsymbol{y} - \boldsymbol{X} \boldsymbol{w}\|_1
$$

## Solution 5
Denote $\alpha_n(\boldsymbol{w}^{(t)})=|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}^{(t)}|^{-1}$ with $\alpha_n(\boldsymbol{w}^{(0)})=1$, the $t$-th iteration of IRLS is given by
$$
    \begin{align*}
        \boldsymbol{w}^{(t+1)}&=\arg\min\limits_{\boldsymbol{w}}\sum\limits_{n=1}^{N}\alpha_n(\boldsymbol{w}^{(t)})|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}|^2
    \end{align*}
$$
Denote $\boldsymbol{\alpha}(\boldsymbol{w}^{(t)})=\begin{bmatrix}
        \alpha_1(\boldsymbol{w}^{(t)})\\
        \alpha_2(\boldsymbol{w}^{(t)})\\
        \vdots\\
        \alpha_N(\boldsymbol{w}^{(t)})
    \end{bmatrix}$, where $\alpha_n(\boldsymbol{w}^{(t)})$ is the $t$-th iteration of $n$-th dimension, we can write the objective function as
$$
    \begin{align*}
        \boldsymbol{w}^{(t+1)}&=\arg\min\limits_{\boldsymbol{w}}\sum\limits_{n=1}^{N}\left(\sqrt{\alpha_n(\boldsymbol{w}^{(t)})}\right)^2|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}|^2\\
        &=\arg\min\limits_{\boldsymbol{w}}\|\mathrm{diag}\left(\boldsymbol{\alpha}(\boldsymbol{w}^{(t)})\right)^{\frac{1}{2}}\cdot(\boldsymbol{y}-\boldsymbol{Xw})\|_2^2=\arg\min\limits_{\boldsymbol{w}}\|(\boldsymbol{A}^{(t)})^{\frac{1}{2}}(\boldsymbol{y}-\boldsymbol{Xw})\|_2^2
    \end{align*}
$$
where $\boldsymbol{A}^{(t)}=\mathrm{diag}(\boldsymbol{\alpha}(\boldsymbol{w}^{(t)}))=\begin{bmatrix}
        \dfrac{1}{|y_1-\boldsymbol{x}_1^\top \boldsymbol{w}^{(t)}|}&  \cdots & 0\\
        \vdots & \ddots & \vdots\\
        0 &  \cdots & \dfrac{1}{|y_N-\boldsymbol{x}_N^\top \boldsymbol{w}^{(t)}|}
    \end{bmatrix}$. To derive the closed-form iterative steps of IRLS, we first denote 
$$    
\begin{align*}
    L&=\|(\boldsymbol{A}^{(t)})^{\frac{1}{2}}(\boldsymbol{y}-\boldsymbol{Xw})\|=(\boldsymbol{y}^\top-w^\top \boldsymbol{X}^\top)((\boldsymbol{A}^{(t)})^{\frac{1}{2}})^\top (\boldsymbol{A}^{(t)})^{\frac{1}{2}}(\boldsymbol{y}-\boldsymbol{Xw})\\
    &=(\boldsymbol{y}^\top-w^\top \boldsymbol{X}^\top)\boldsymbol{A}^{(t)}(\boldsymbol{y}-\boldsymbol{Xw})\\
    &=\boldsymbol{y}^\top \boldsymbol{A}^{(t)}\boldsymbol{y}-\boldsymbol{y}^\top \boldsymbol{A}^{(t)}\boldsymbol{Xw}-\boldsymbol{w}^\top \boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{y}+\boldsymbol{w}^\top \boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{Xw}\\
    \dfrac{\partial L}{\partial \boldsymbol{w}}&=-2\boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{y}+2\boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{Xw}=0\\
    \Rightarrow \boldsymbol{w}^{(t+1)}&=(\boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{X})^{-1}\boldsymbol{X}^\top \boldsymbol{A}^{(t)}\boldsymbol{y}
\end{align*}
$$
Since we know $\boldsymbol{w}^{(t)}$ and $\boldsymbol{A}^{(t)}$, we can calculate $\boldsymbol{w}^{(t+1)}$ iteratively.

**Explanation:** 
> The idea of IRLS is to simply treat $\boldsymbol{w}^*=\arg\min\limits_{\boldsymbol{w}}\sum\limits_{n=1}^{N}|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}|^2$, but with a weight $\alpha_n(\boldsymbol{w}^{(t)})$ attached to each $n$. The initial weights are set as $\boldsymbol{\alpha}=\boldsymbol{1}$. \textbf{In each iteration, we can lower an observation's importance (i.e. weight) by $\alpha_n(\boldsymbol{w}^{(t)})=|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}^{(t)}|^{-1}$ if it has a large residual (i.e. outliners).}

> Naturally, IRLS works for p-norm with $p<2$: $\alpha_n^{(t)}=|y_n-\boldsymbol{x}_n^\top \boldsymbol{w}^{(t)}|^{p-2}$.

In [52]:
def training_irls(x: np.ndarray,
                  y: np.ndarray,
                  iteration: int = 100,
                  r_seed: int = 1) -> np.ndarray:
    """
    min_w ||y - Xw||_1 (MAE minimization)

    :param x: testing data with size (N, dim)
    :param y: testing label with size (N, 1)
    :param iteration: the number of iterations for soft-thresholding
    :param r_seed: the random seed for initializing model.
    :return:
        a weight vector with size (dim, 1)
    """
    np.random.seed(r_seed)
    N = x.shape[0]
    dim = x.shape[1]
    a = np.ones(N)
    w = np.random.randn(dim, 1)
    for t in range(iteration):
        # Update vector a^(t) and construct matrix A^(t)
        for n in range(N):
            a[n] = 1 / abs(y[n] - x[n].T @ w)
        A = np.diag(a)
        # Update w^(t+1)
        w = np.linalg.inv(x.T @ A @ x) @ x.T @ A @ y
    return w

In [53]:
x = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([[3], [7], [11]])
# this should be [[1], [1]] because y = x1 + x2
weights = training_irls(x, y, iteration=10000)
weights

array([[1.],
       [1.]])

# Test Script

In [56]:
data1 = linear_data_simulator(prior='Gauss', noise='Gauss')
data2 = linear_data_simulator(prior='Laplace', noise='Gauss')
data3 = linear_data_simulator(prior='Gauss', noise='Laplace')
w1 = training(data1['train'][0], data1['train'][1], gamma=1)
w2 = training_sgd(data1['train'][0], data1['train'][1], gamma=1)
w3 = training_lasso(data2['train'][0], data2['train'][1], gamma=1)
w4 = training_elastic(
    data2['train'][0], data2['train'][1], gamma1=1, gamma2=1)
w5 = training_irls(data3['train'][0], data3['train'][1])

mse_w1 = mse(data1['real'], w1)
mse_w2 = mse(data1['real'], w2)
mse_w3 = mse(data2['real'], w3)
mse_w4 = mse(data2['real'], w4)
mse_w5 = mse(data3['real'], w5)

mse_y1 = testing(data1['test'][0], data1['test'][1], w1)
mse_y2 = testing(data1['test'][0], data1['test'][1], w2)
mse_y3 = testing(data2['test'][0], data2['test'][1], w3)
mse_y4 = testing(data2['test'][0], data2['test'][1], w4)
mse_y5 = testing(data3['test'][0], data3['test'][1], w5)

print(
    'Ridge regression, mse-w={:.4f}, mse-y={:.4f}'.format(mse_w1, mse_y1))
print(
    'Ridge regression (SGD), mse-w={:.4f}, mse-y={:.4f}'.format(mse_w2, mse_y2))
print(
    'Lasso regularizer, mse-w={:.4f}, mse-y={:.4f}'.format(mse_w3, mse_y3))
print(
    'Elastic Net regularizer, mse-w={:.4f}, mse-y={:.4f}'.format(mse_w4, mse_y4))
print('IRLS, mse-w={:.4f}, mse-y={:.4f}'.format(mse_w5, mse_y5))


Ridge regression, mse-w=0.5294, mse-y=2.6081
Ridge regression (SGD), mse-w=1.8516, mse-y=18.4436
Lasso regularizer, mse-w=0.7350, mse-y=2.4981
Elastic Net regularizer, mse-w=0.5270, mse-y=2.7996
IRLS, mse-w=0.9128, mse-y=5.2916
